# Test inference on ESP32 microcontroller

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import serial
import time

import numpy as np
import tensorflow as tf

from PIL import Image
import matplotlib.pyplot as plt

In [2]:
try:
  %tensorflow_version 2.x
except:
  pass

In [3]:
INPUT_IMAGE_SIZE = 14

# Path to TensorFlow Lite model file
tflite_model_file = 'model.tflite'

# Path to images folder
path = 'test_images_14x14/'
file_format = ".jpg"

SERIAL_PORT_NAME = '/dev/ttyUSB0'

EXPECTED_PRECISION = 0.000001

In [4]:
def init_serial(port_name):
    # Configure and open serial port
    port = serial.Serial(
        port=port_name,
        baudrate=115200,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        xonxoff=False,
        rtscts=False,
        dsrdtr=False
    )
    
    port.reset_input_buffer()
    port.reset_output_buffer()
    return port

In [5]:
def send_to_mcu(port, data):
    bytes_written = port.write(data)
    return bytes_written

In [6]:
def read_result_from_mcu(port):
    line = port.read_until()   # read a '\n' terminated line
    return line

In [7]:
def infer_with_TF_lite(interpreter, input_details, output_details, raw_image):
    # Get input size
    input_shape = input_details[0]['shape']
    size = input_shape[:2] if len(input_shape) == 3 else input_shape[1:3]

    # Preprocess image
    raw_image = raw_image.resize(size)
    img = np.array(raw_image, dtype=np.float32)

    # Normalize image
    img = img / 255.

    # Add a batch dimension and a dimension because we use grayscale format
    # Reshape from (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE) to (1, INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 1)
    input_data = img.reshape(1, img.shape[0], img.shape[1], 1)
    
    # Point the data to be used for testing
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the interpreter
    interpreter.invoke()

    # Obtain results
    predictions = interpreter.get_tensor(output_details[0]['index'])[0]
    
    return predictions

In [8]:
def infer_with_MCU(raw_image):
    img = np.array(raw_image, dtype=np.uint8)
    bytes_sent = send_to_mcu(serial_port, img)
    
    response_str = read_result_from_mcu(serial_port)
    response_str = response_str.decode("utf-8")
    predictions = np.fromstring(response_str, dtype=np.float32, sep=',')
    return predictions

In [9]:
def compare_results(result_tfl, result_mcu):
    for i in range(len(result_tfl)):
        if abs(result_tfl[i] - result_mcu[i]) > EXPECTED_PRECISION :
            return False
        else:
            pass
    return True

In [10]:
# Create Interpreter (Load TFLite model).
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
# Allocate tensors
interpreter.allocate_tensors()

In [11]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

### Be sure your ESP32 board connected to development computer.

In [12]:
# Get list of present serial ports
!dmesg | grep -e tty

[    0.000000] console [tty0] enabled
[    1.229250] tty ttyS17: hash matches
[  438.474326] usb 1-1: cp210x converter now attached to ttyUSB0


In [13]:
# Configure and open serial port
serial_port = init_serial(SERIAL_PORT_NAME)

# Check which port was really used
print("Opened serial port : {0}".format(serial_port.name))        

Opened serial port : /dev/ttyUSB0


In [14]:
files = [files for files in glob.glob(path + "*" + file_format, recursive=False)]
print("Found {0} {1} files".format(len(files), file_format))

Found 50 .jpg files


### Run inference on TensorFlow Lite model and on MCU. Compare results.

In [15]:
for f in files:
    # Read image
    raw_image = Image.open(f).convert('L')
    
    result_tfl = infer_with_TF_lite(interpreter, input_details, output_details, raw_image)

    result_mcu = infer_with_MCU(raw_image)

    status = compare_results(result_tfl, result_mcu)
    status_str = "Ok" if status else "Reults NOT eqal!"
    print("For file: {0} - {1}".format(f, status_str))

For file: test_images_14x14/shirt_9.jpg - Ok
For file: test_images_14x14/coat_8.jpg - Ok
For file: test_images_14x14/trouser_14.jpg - Ok
For file: test_images_14x14/dress_12.jpg - Ok
For file: test_images_14x14/trouser_0.jpg - Ok
For file: test_images_14x14/shirt_33.jpg - Ok
For file: test_images_14x14/ankle boot_49.jpg - Ok
For file: test_images_14x14/bag_29.jpg - Ok
For file: test_images_14x14/sneaker_44.jpg - Ok
For file: test_images_14x14/dress_46.jpg - Ok
For file: test_images_14x14/pullover_23.jpg - Ok
For file: test_images_14x14/coat_27.jpg - Ok
For file: test_images_14x14/trouser_2.jpg - Ok
For file: test_images_14x14/pullover_36.jpg - Ok
For file: test_images_14x14/dress_11.jpg - Ok
For file: test_images_14x14/dress_45.jpg - Ok
For file: test_images_14x14/trouser_30.jpg - Ok
For file: test_images_14x14/shirt_6.jpg - Ok
For file: test_images_14x14/sandal_20.jpg - Ok
For file: test_images_14x14/sandal_32.jpg - Ok
For file: test_images_14x14/bag_15.jpg - Ok
For file: test_images_

In [16]:
serial_port.close()